*Goal: Compare standard k-means clustering vs Time Series k-means clustering*

Clustering is an unsupervised learning task where an algorithm groups similar data points with no 'ground truth' labels. 
Similarity between data points is measured with a distance metric, commonly Euclidean distance.
Clustering different **time series** into similar groups is a challenging clustering task because **each data point is an ordered sequence**.

The distance measures used in standard clustering algorithms, such as Euclidean distance, are often not appropriate to time series as it is invariant to time shifts. 
A better approach is to replace the default distance measure with a metric for comparing time series, such as **Dynamic Time Warping** 

[Explaination of DTW](http://alexminnaar.com/2014/04/16/Time-Series-Classification-and-Clustering-with-Python.html)
Dynamic time warping finds the optimal non-linear alignment between two time series.
The Euclidean distances between alignments are then much less susceptible to pessimistic similarity measurements due to distortion in the time axis. There is a price to pay for this, however, because dynamic time warping is **quadratic in the length of the time series used**.

DTW is calculated as the **squared root of the sum of squared distances between each element in X and its nearest point in Y**. *Note that DTW(X, Y) ≠ DTW(Y, X)*

With DTW, the centroids have an average shape that mimics the shape of the members of the cluster, regardless of where temporal shifts occur amongst the members.

[dtw-kmeans](https://towardsdatascience.com/how-to-apply-k-means-clustering-to-time-series-data-28d04a8f7da3)
*Note that tslearn expects a single time series to be formatted as two-dimensional array. A set of time series should be formatted as a three-dimensional array with shape (num_series, max_length, 1)*

kmeans vs gmm vs dtw kmeans: then i can show that gmm gives decent clusters in a decent time frame so its the one i will choose to use from now on

In [1]:
import sys
sys.path.append('C:\\Users\\sumaiyah\\Documents\\Univeristy\\diss\\misuse-AIaaS')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from timeit import default_timer as timer
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

In [2]:
# Import filepaths
from notebooks import FIC_60minute_filepath

In [3]:
seed = 0

In [4]:
fic_df = pd.read_pickle(FIC_60minute_filepath(1)).drop(columns=[0,1,2,3])#.head(1000)

In [5]:
fic_df = TimeSeriesScalerMeanVariance().fit_transform(fic_df)

In [6]:
k=9
clusters = pd.DataFrame()

In [7]:
from tslearn.utils import to_time_series_dataset

# Need values in specific shape for DTW
dtw_vals = to_time_series_dataset(fic_df)
print(dtw_vals.shape)

sz = dtw_vals.shape[1]

(46412, 24, 1)


# Dynamic Time Warping

In [ ]:
print("DTW k-means k=%d" % k)

start = timer()
model = TimeSeriesKMeans(n_clusters=k,
                          metric="dtw",
                          verbose=False,
                          random_state=42,
                          n_jobs=3)
dtw_kmeans_predictions = model.fit_predict(dtw_vals)
end = timer()
print('Clustering k=%d took %f seconds' % (k, (end - start)))

In [ ]:
clusters['dtw_label'] = dtw_kmeans_predictions
print(clusters['dtw_label'].value_counts(), '\n')

# K-means

In [8]:
# Euclidean k-means
print("Euclidean k-means")

start = timer()

km = TimeSeriesKMeans(n_clusters=k, verbose=False, random_state=42)
kmeans_predictions = km.fit_predict(dtw_vals)

end = timer()
print('Clustering k=%d took %f seconds' % (k, (end - start)))

clusters['kmeans_label'] = kmeans_predictions
print(clusters['kmeans_label'].value_counts(), '\n')

Euclidean k-means
Clustering k=9 took 42.219677 seconds
0    23195
2     5813
6     5096
1     3135
8     2147
4     2031
7     1797
5     1753
3     1445
Name: kmeans_label, dtype: int64 



# GMM

In [9]:
from sklearn.mixture import GaussianMixture
   
print('Results of GMM Clustering with k=%d' % k)  

start = timer()
    
# training gaussian mixture model 
gmm = GaussianMixture(n_components=k, random_state=42, init_params='random')
gmm.fit(dtw_vals[:,:,0])
gmm_labels = gmm.predict(dtw_vals[:,:,0])
    
end = timer()

print('Clustering k=%d took %f seconds' % (k, (end - start)))
clusters['gmm_label'] = gmm_labels
print(clusters['gmm_label'].value_counts(), '\n')

Results of GMM Clustering with k=9
Clustering k=9 took 19.359882 seconds
7    13268
6     9550
5     5573
4     4233
8     3455
1     2918
0     2804
3     2743
2     1868
Name: gmm_label, dtype: int64 



# Results

# DTW Kmeans

In [11]:
# # Print each cluster in order of size from largest cluster to smallest
# cluster_order = pd.Series(dtw_kmeans_predictions).value_counts().keys().tolist()

# fig, ax = plt.subplots(3, 3, figsize=(12, 10), sharex=True, sharey=True)

# fig.text(0.5, 0, 'Time (hours)', ha='center', fontsize=14)
# fig.text(0, 0.5, 'ln(function invocation count)', va='center', rotation='vertical', fontsize=14)
# fig.suptitle("DTW $k$-means")


# for yi in range(len(cluster_order)):
#     row, col = yi // 3, yi % 3
#     cluster_index = cluster_order[yi]
    
#     for xx in dtw_vals[dtw_kmeans_predictions == yi]:
#         ax[row, col].plot(xx.ravel(), "k-", alpha=.05)
#         ax[row, col].plot(model.cluster_centers_[cluster_index].ravel(), "r-")
        
#     ax[row, col].text(16, -4.5, 
#                       'Cluster %d \n size:%d' % ((yi + 1), pd.Series(dtw_kmeans_predictions).value_counts().values[yi]),
#                       color="#95190C",
#                       fontsize=13)
    
# plt.tight_layout()
# plt.savefig('dtwkm.png', dpi=500)
# plt.show()

# k-means

In [ ]:
# Print each cluster in order of size from largest cluster to smallest
cluster_order = pd.Series(kmeans_predictions).value_counts().keys().tolist()

fig, ax = plt.subplots(3, 3, figsize=(12, 10), sharex=True, sharey=True)

fig.text(0.5, 0, 'Time (hours)', ha='center', fontsize=14)
fig.text(0, 0.5, 'ln(function invocation count)', va='center', rotation='vertical', fontsize=14)
fig.suptitle("Euclidian $k$-means")

for yi in range(len(cluster_order)):
    row, col = yi // 3, yi % 3
    cluster_index = cluster_order[yi]
    
    for xx in dtw_vals[kmeans_predictions == cluster_index]:
        ax[row, col].plot(xx.ravel(), "k-", alpha=.05)
        ax[row, col].plot(km.cluster_centers_[cluster_index].ravel(), "r-")

    ax[row, col].text(16, -4.5, 
                      'Cluster %d \nSize:%d' % ((yi + 1), pd.Series(kmeans_predictions).value_counts().values[yi]),
                      color="#95190C",
                      fontsize=13, 
                      weight='bold')

plt.tight_layout()
plt.savefig('kmm.png', dpi=500)
plt.show()

# GMM

In [ ]:
import scipy.stats

# https://stackoverflow.com/questions/47412749/how-can-i-get-a-representative-point-of-a-gmm-cluster
centers = np.empty(shape=(gmm.n_components, dtw_vals[:,:,0].shape[1]))
for i in range(gmm.n_components):
    density = scipy.stats.multivariate_normal(cov=gmm.covariances_[i], mean=gmm.means_[i]).logpdf(dtw_vals[:,:,0])
    centers[i, :] = dtw_vals[:,:,0][np.argmax(abs(density))]

# Print each cluster in order of size from largest cluster to smallest
cluster_order = pd.Series(gmm_labels).value_counts().keys().tolist()

fig, ax = plt.subplots(3, 3, figsize=(12, 10), sharex=True, sharey=True)

fig.text(0.5, 0, 'Time (hours)', ha='center', fontsize=14)
fig.text(0, 0.5, 'ln(function invocation count)', va='center', rotation='vertical', fontsize=14)
fig.suptitle("Gaussian mixture model")

for yi in range(len(cluster_order)):
    row, col = yi // 3, yi % 3
    cluster_index = cluster_order[yi]
    
    for xx in dtw_vals[gmm_labels == cluster_index]:
        ax[row, col].plot(xx.ravel(), "k-", alpha=.01)
        ax[row, col].plot(centers[cluster_index].ravel(), "r-")

    ax[row, col].text(16, -4.5, 
                      'Cluster %d \nSize:%d' % ((yi + 1), pd.Series(gmm_labels).value_counts().values[yi]),
                      color="#95190C",
                      fontsize=13, 
                      weight='bold')

plt.tight_layout()
plt.savefig('gmm.png', dpi=500)
plt.show()

| Clustering Algorithm           | Cluster Sizes                                           | Clustering Time (s) |
|--------------------------------|---------------------------------------------------------|---------------------|
| Gaussian Mixture Model         | [13268, 9550, 5573, 4233, 3455, 2918, 2804, 2743, 1868] | 16.121              |
| Euclidean $K$-means            | [23195, 5813, 5096, 3135, 2147, 2031, 1797, 1753, 1445] | 53.384              |
| Dynamic Time Warping $K$-means | [11970, 11740, 6175, 5048, 3627, 3064, 2159, 1753, 876] | 1876.661            |